# Klavis Sandbox

This notebook demonstrates how to run Klavis Sandbox environment and MCP Servers (remote + local) against a complicated task


In [ ]:
%pip install -q klavis python-dotenv langchain langchain-mcp-adapters langchain-anthropic

In [ ]:
import json
import os
from dotenv import load_dotenv
from langchain_anthropic import ChatAnthropic
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import create_agent
from klavis import Klavis, SandboxMcpServer

load_dotenv()

CURRENT_DIR = os.path.dirname(os.path.abspath("__file__")) if os.path.dirname(os.path.abspath("__file__")) else os.getcwd()
LOCAL_MCP_SERVERS_PATH = os.path.abspath(os.path.join(CURRENT_DIR, "..", "..", "mcp_servers", "local"))

klavis_client = Klavis(api_key=os.getenv("KLAVIS_API_KEY"))

print(f"Current Directory: {CURRENT_DIR}")
print(f"MCP Servers Local Path: {LOCAL_MCP_SERVERS_PATH}")

### Step 1: Create Sandboxes environment for Gmail and Snowflake

In [ ]:
# Create Gmail sandbox
gmail_sandbox = klavis_client.sandbox.create_sandbox(
    server_name=SandboxMcpServer.GMAIL,
)

snowflake_sandbox = klavis_client.sandbox.create_sandbox(
    server_name=SandboxMcpServer.SNOWFLAKE,
)

print(f"Gmail Sandbox: {gmail_sandbox}")
print(f"Snowflake Sandbox: {snowflake_sandbox}")

In [ ]:
# check existing environment in gmail sandbox
response = klavis_client.sandbox.dump_sandbox(
    sandbox_id=gmail_sandbox.sandbox_id,
)

print(response)

In [ ]:
# check existing environment in snowflake sandbox
response = klavis_client.sandbox.dump_sandbox(
    sandbox_id=snowflake_sandbox.sandbox_id,
)

print(json.dumps(response.data, indent=2))

### Step 2: Initialize Snowflake Sandbox with Data

In [ ]:
snowflake_json_path = os.path.join(CURRENT_DIR, "snowflake.json")

with open(snowflake_json_path, "r") as f:
    snowflake_data = json.load(f)
    
klavis_client.sandbox.initialize_sandbox(
    sandbox_id=snowflake_sandbox.sandbox_id,
    databases=snowflake_data["databases"],
)

In [ ]:
# after loading data, check environment in snowflake sandbox
response = klavis_client.sandbox.dump_sandbox(
    sandbox_id=snowflake_sandbox.sandbox_id,
)

print(json.dumps(response.data, indent=2))

### Step 3: Setup MCP Client, AI Agent and kick off task

In [ ]:
# Create MCP client with remote and local MCP Servers
mcp_client = MultiServerMCPClient({
    "snowflake": {
        "transport": "streamable_http",
        "url": snowflake_sandbox.server_url,
    },
    "gmail": {
        "transport": "streamable_http",
        "url": gmail_sandbox.server_url,
    },
    "filesystem": {
        "transport": "stdio",
        "command": "npx",
        "args": [
            "-y",
            os.path.join(LOCAL_MCP_SERVERS_PATH, "filesystem"),
            CURRENT_DIR
        ],
        "env": dict(os.environ)
    },
    "pdf-tools": {
        "transport": "stdio",
        "command": "uvx",
        "args": [
            "--from",
            os.path.join(LOCAL_MCP_SERVERS_PATH, "pdf-tools"),
            "pdf-tools-mcp",
            "--workspace_path",
            CURRENT_DIR,
            "--tempfile_dir",
            CURRENT_DIR
        ],
        "env": dict(os.environ)
    },
})

tools = await mcp_client.get_tools()
llm = ChatAnthropic(model="claude-sonnet-4-5-20250929")

agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=(
        "You are an intelligent SLA Monitoring Agent. Your goal is to ensure compliance with Service Level Agreements (SLAs) and manage communications for breaches.\n"
        "You have access to below tools:\n"
        "1. Snowflake: To fetch user and ticket data.\n"
        "2. PDF/Filesystem: To read SLA manuals and email templates from files of the local filesystem.\n"
        "3. Gmail: to send emails if needed.\n\n"
        "General Workflow:\n"
        "1. Discovery: Explore the database to understand the data schema (users, tickets, etc.).\n"
        "2. SLA Extraction: Read the provided SLA manual (PDF) to determine response time limits for different service levels.\n"
        "3. Analysis: Compare the data against the SLA rules. Calculate elapsed time since creation.\n"
        "4. Action: For any breaches:\n"
        "   - Find the appropriate email template.\n"
        "   - Send a reminder to the manager.\n"
        "   - Send an apology to the user.\n\n"
        "Always verify the current date/time when calculating SLAs."
    ),
)

In [ ]:
# print tools
for i, tool in enumerate(tools, 1):
    print(f"{i}. {tool.name}")

In [ ]:
user_message = (
    "Identify the tickets in the database that have exceeded the initial response time according to the relevant documentation, and send "
    "reminder emails, based on the templates mentioned in the manual, to the respective responsible managers, as well as apology emails to all involved users."
)

print("🚀 Starting agent...\n")

async for event in agent.astream_events(
    {"messages": [{"role": "user", "content": user_message}]},
    version="v2",
):
    kind = event["event"]
    
    if kind == "on_chat_model_stream":
        chunk = event["data"]["chunk"]
        # Try to extract content from the chunk
        if hasattr(chunk, "content"):
            content = chunk.content
            if content:
                # Handle both string and list content
                if isinstance(content, list):
                    for item in content:
                        if isinstance(item, dict) and "text" in item:
                            print(item["text"], end="", flush=True)
                        elif hasattr(item, "text"):
                            print(item.text, end="", flush=True)
                elif isinstance(content, str):
                    print(content, end="", flush=True)
    
    elif kind == "on_tool_start":
        tool_name = event["name"]
        tool_input = {k: v for k, v in event["data"].get("input", {}).items() 
                      if k not in ["runtime", "config", "stream_writer", "tool_call_id", "store"]}
        print(f"\n🔧 {tool_name}")
        if tool_input:
            print(f"   → {tool_input}")
    
    elif kind == "on_tool_end":
        print("\n")
        pass

print("\n\n✨ Done!")

### Step 5: Cleanup - Delete Sandboxes

In [ ]:
klavis_client.sandbox.delete_sandbox(
    server_name=SandboxMcpServer.GMAIL,
    sandbox_id=gmail_sandbox.sandbox_id,
)

klavis_client.sandbox.delete_sandbox(
    server_name=SandboxMcpServer.SNOWFLAKE,
    sandbox_id=snowflake_sandbox.sandbox_id,
)